# BestEthernet

BestEthernetは、Pythonを使用してイーサネット接続の速度を測定し、最も高速な接続を自動的に選択するツールです。USBテザリングを有効にし、利用可能なイーサネット接続の中から最速のものを選択してアクティブにします。

## 機能

- USBテザリングの有効化
- 利用可能なイーサネット接続のリストアップ
- 各イーサネット接続のダウンロードおよびアップロード速度の測定
- 最速のイーサネット接続を自動的に選択してアクティブにする
- 速度測定結果のログ記録

イーサネット一覧:
 
Name   Status InterfaceDescription              
----   ------ --------------------              
hotel  Up     Realtek PCIe GbE Family Controller
イーサネット Up     Remote NDIS Compatible Device #2  



ダウンロード速度: 0.1 Mbps, アップロード速度: 0.8 Mbps
2024-03-02 20:06:47: イーサネットに更新しました。ダウンロード速度: 0.1 Mbps, アップロード速度: 0.8 Mbps



In [8]:
import subprocess

def enable_usb_tethering(adb_path):
    subprocess.run([adb_path, "shell", "svc", "wifi", "disable"], capture_output=True)
    subprocess.run([adb_path, "shell", "svc", "data", "enable"], capture_output=True)
    subprocess.run([adb_path, "shell", "service call connectivity 34 i32 1"], capture_output=True)

def update_main_ethernet(interface_list):
    if fastest_interface:
        print(f"最速のインターフェースは '{fastest_interface}' です。このインターフェースをアクティブにします。")
        # ここで最速のインターフェースをアクティブにする処理を追加します。
        # Windowsの場合、PowerShellコマンドを使用することが考えられます。
        subprocess.run(["powershell", "-Command", f"Enable-NetAdapter -Name '{fastest_interface}'"], capture_output=True)
        

adb_path = r".\..\AndroidDebugTools\platform-tools\adb.exe"

enable_usb_tethering(adb_path)

# 2

In [9]:
import subprocess

def get_ethernet_interfaces():
    # すべてのネットワークアダプタを取得
    result = subprocess.run(["powershell", "-Command", "Get-NetAdapter | Select-Object -ExpandProperty Name"], capture_output=True, text=True)
    interfaces = result.stdout.strip().split('\n')
    return interfaces

def enable_all_ethernet_interfaces(interfaces):
    for interface in interfaces:
        # すべてのインターフェースを有効にする
        subprocess.run(["powershell", "-Command", f"Enable-NetAdapter -Name \"{interface}\" -Confirm:$false"], capture_output=True, text=True)
        print(f"{interface}を有効にしました。")

def main():
    interfaces = get_ethernet_interfaces()
    print("全てのイーサネットインターフェース:", interfaces)
    enable_all_ethernet_interfaces(interfaces)

if __name__ == "__main__":
    main()

全てのイーサネットインターフェース: ['hotel', 'router', 'VPN - VPN Client', 'iPhone']
hotelを有効にしました。
routerを有効にしました。
VPN - VPN Clientを有効にしました。
iPhoneを有効にしました。


In [13]:
import subprocess
import speedtest
import datetime
import csv
import time

def get_ethernet_interfaces():
    result = subprocess.run(["powershell", "-Command", "Get-NetAdapter | Where-Object {$_.Status -eq 'Up'} | Select-Object -ExpandProperty Name"], capture_output=True, text=True)
    interfaces = result.stdout.strip().split('\n')
    return interfaces


def enable_all_ethernet_interfaces(interfaces):
    """
    与えられたすべてのイーサネットインターフェースを有効にします。
    """
    for interface in interfaces:
        subprocess.run(["powershell", "-Command", f"Enable-NetAdapter -Name \"{interface}\" -Confirm:$false"], capture_output=True, text=True)
        print(f"{interface}を有効にしました。")

def switch_ethernet_interface(interface_name, enable=True):
    """
    指定されたイーサネットインターフェースを有効または無効にします。
    enableパラメータがTrueの場合は有効に、Falseの場合は無効にします。
    """
    action = "Enable-NetAdapter" if enable else "Disable-NetAdapter"
    subprocess.run(["powershell", "-Command", f"{action} -Name \"{interface_name}\" -Confirm:$false"], capture_output=True, text=True)
    state = "有効" if enable else "無効"
    print(f"{interface_name}を{state}にしました。")
    
    # ログファイルにインターフェースの切り替え履歴を記録
    log_message = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: {interface_name}を{state}にしました。\n"
    with open("C:\\Users\\100ca\\Documents\\PyCode\\internet\\log\\change_log.txt", "a", encoding='utf-8') as log_file:
        log_file.write(log_message)

def measure_speed(interface_name):
    """
    指定されたイーサネットインターフェースのインターネット速度（ダウンロードとアップロード）を測定します。
    """
    st = speedtest.Speedtest()
    st.download()
    st.upload()
    download_speed = round(st.results.dict()['download'] / 1_000_000, 1)
    upload_speed = round(st.results.dict()['upload'] / 1_000_000, 1)
    print(f"ダウンロード速度: {download_speed} Mbps, アップロード速度: {upload_speed} Mbps")
    return download_speed, upload_speed

def log_results(interface_name, download_speed, upload_speed):
    """
    インターネット速度の測定結果をログファイルとCSVファイルに記録します。
    """
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"{timestamp}: {interface_name}に更新しました。ダウンロード速度: {download_speed} Mbps, アップロード速度: {upload_speed} Mbps\n"
    print(log_message)
    
    log_file_path = "C:\\Users\\100ca\\Documents\\PyCode\\internet\\log\\change_log.txt"
    csv_file_path = "C:\\Users\\100ca\\Documents\\PyCode\\internet\\log\\log.csv"
    
    with open(log_file_path, "a", encoding='utf-8') as log_file:
        log_file.write(log_message)
    
    with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, interface_name, download_speed, upload_speed])

def main():
    """
    メイン関数では、以下のステップを実行します。
    1. すべてのイーサネットインターフェースを取得します。
    2. 取得したインターフェースを一時的に無効にします。
    3. 各インターフェースを順番に有効にし、インターネット速度を測定します。
    4. 測定結果をログに記録します。
    5. 最後に、すべてのインターフェースを再度有効にします。
    """
    interfaces = get_ethernet_interfaces()
    print("利用可能なイーサネットインターフェース:", interfaces)

#    for interface in interfaces:
#        switch_ethernet_interface(interface, enable=False)
    
    for interface in interfaces:
        switch_ethernet_interface(interface, enable=True)
        print("10秒間停止しています...")
        time.sleep(10)  
        print("測定中")
        download_speed, upload_speed = measure_speed(interface)
        log_results(interface, download_speed, upload_speed)
        switch_ethernet_interface(interface, enable=False)
    
    # 最後にすべてのインターフェースを再度有効にする
    for interface in interfaces:
        switch_ethernet_interface(interface, enable=True)

if __name__ == "__main__":
    main()

利用可能なイーサネットインターフェース: ['hotel', 'iPhone']
hotelを有効にしました。
10秒間停止しています...
測定中
ダウンロード速度: 24.1 Mbps, アップロード速度: 12.0 Mbps
2024-03-02 21:59:22: hotelに更新しました。ダウンロード速度: 24.1 Mbps, アップロード速度: 12.0 Mbps

hotelを無効にしました。
iPhoneを有効にしました。
10秒間停止しています...
測定中
ダウンロード速度: 24.9 Mbps, アップロード速度: 11.8 Mbps
2024-03-02 21:59:57: iPhoneに更新しました。ダウンロード速度: 24.9 Mbps, アップロード速度: 11.8 Mbps

iPhoneを無効にしました。
hotelを有効にしました。
iPhoneを有効にしました。


# 最速のやつに切り替える（上手くいっていない）

In [32]:
def update_main_ethernet(interfaces):
    import csv
    from collections import defaultdict
    import subprocess

    # CSVファイルから速度測定結果を読み込み、インターフェースごとに速度を格納
    speed_results = defaultdict(list)
    with open("C:\\Users\\100ca\\Documents\\PyCode\\internet\\log\\log.csv", mode='r', encoding='cp932', errors='ignore') as file:
        reader = csv.DictReader(file)
        for row in reader:
            interface_name = row['EthernetName']
            download_speed = 0
            upload_speed = 0
            if row['Download'] is not None and row['Download'].replace('.', '', 1).isdigit():
                download_speed = float(row['Download'])
            if row['Upload'] is not None and row['Upload'].replace('.', '', 1).isdigit():
                upload_speed = float(row['Upload'])
            speed_results[interface_name].append((download_speed, upload_speed))

    # 各インターフェースの最新1件の速度測定結果を用いて平均速度を計算
    average_speeds = {}
    for interface, speeds in speed_results.items():
        recent_speeds = speeds[-3:] if len(speeds) > 3 else speeds
        average_download_speed = sum(download for download, _ in recent_speeds) / len(recent_speeds)
        average_speeds[interface] = average_download_speed
#print results

    # 最速のインターフェースを特定
    fastest_interface = max(average_speeds, key=average_speeds.get)

    print(f"最速のインターフェースは '{fastest_interface}' です。このインターフェースをアクティブにします。")
    # 最速のインターフェースをアクティブにする
    subprocess.run(["powershell", "-Command", f"Enable-NetAdapter -Name '{fastest_interface}'"], capture_output=True)

In [33]:
interfaces = get_ethernet_interfaces()
update_main_ethernet(interfaces)

最速のインターフェースは 'hotel' です。このインターフェースをアクティブにします。
